# Data Extraction - Azure OpenAI GPT-4o with Vision

This sample demonstrates how to extract structured data from any document using Azure OpenAI's GPT-4o model with vision capabilities.

![Data Extraction](../../../images/extraction-openai.png)

This is achieved by the following process:

- Construct a system prompt that defines the instruction for extracting structured data from documents.
- Construct a user prompt that includes the specific extraction instruction for the type of document, and each document page as a base64 encoded image.
- Use the Azure OpenAI chat completions API with the GPT-4o model to generate a structured output from the content.

## Objectives

By the end of this sample, you will have learned how to:

- Convert a document into a set of base64 encoded images for processing by GPT-4o.
- Use prompt engineering techniques to instruct GPT-4o to extract structured data from a type of document.
- Use the [Structured Outputs feature](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/structured-outputs?tabs=python-secure) to extract structured data from the document page images using Azure OpenAI's GPT-4o model.
- Use the [logprobs](https://learn.microsoft.com/en-us/azure/ai-services/openai/reference#request-body:~:text=False-,logprobs,-integer) parameter in an OpenAI request to determine the confidence of the extracted structured output.

## Setup

### Import modules

This sample takes advantage of the following Python dependencies:

- **pdf2image** for converting a PDF file into a set of images per page.
- **openai** to interface with the Azure OpenAI chat completions API to generate structured extraction outputs using the GPT-4o model.
- **azure-identity** to securely authenticate with deployed Azure Services using Microsoft Entra ID credentials.

The following local modules are also used:

- **modules.app_settings** to access environment variables from the `.env` file.
- **modules.comparison** to compare the output of the extraction process with expected results.
- **modules.document_processing_result** to store the results of the extraction process as a file.
- **modules.openai_confidence** to calculate the confidence of the classification process based on the `logprobs` response from the API request.
- **modules.vehicle_insurance_policy** to provide the expected structured output JSON schema for vehicle insurance policy documents.
- **modules.utils** `Stopwatch` to measure the end-to-end execution time for the classification process.

In [1]:
import sys
sys.path.append('../../') # Import local modules

from IPython.display import display
import os
import pandas as pd
from dotenv import dotenv_values
from pdf2image import convert_from_bytes
import base64
import io
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import json

from modules.app_settings import AppSettings
from modules.utils import Stopwatch
from modules.accuracy_evaluator import AccuracyEvaluator
from modules.comparison import get_extraction_comparison
from modules.openai_confidence import evaluate_confidence
from modules.vehicle_insurance_policy import VehicleInsurancePolicy
from modules.document_processing_result import DataExtractionResult

### Configure the Azure services

To use Azure OpenAI, the SDK is used to create a client instance using a deployed endpoint and authentication credentials.

For this sample, the credentials of the Azure CLI are used to authenticate with the deployed services.

In [2]:
# Set the working directory to the root of the repo
working_dir = os.path.abspath('../../../')
settings = AppSettings(dotenv_values(f"{working_dir}/.env"))

# Configure the default credential for accessing Azure services using Azure CLI credentials
credential = DefaultAzureCredential(
    exclude_workload_identity_credential=True,
    exclude_developer_cli_credential=True,
    exclude_environment_credential=True,
    exclude_managed_identity_credential=True,
    exclude_powershell_credential=True,
    exclude_shared_token_cache_credential=True,
    exclude_interactive_browser_credential=True
)

openai_token_provider = get_bearer_token_provider(credential, 'https://cognitiveservices.azure.com/.default')

openai_client = AzureOpenAI(
    azure_endpoint=settings.openai_endpoint,
    azure_ad_token_provider=openai_token_provider,
    api_version="2024-12-01-preview" # Requires the latest API version for structured outputs.
)

### Establish the expected output

To compare the accuracy of the extraction process, the expected output of the extraction process has been defined in the following code block based on each page of a [Vehicle Insurance Policy](../../assets/vehicle_insurance/policy_1.pdf).

> **Note**: More insurance policy examples can be found in the [assets folder](../../assets/vehicle_insurance). These examples include the PDF file and an associated JSON metadata file that provides the expected structured output. You can add your own scenarios by following the same structure.

The expected output has been defined by a human evaluating the document.

In [3]:
path = f"{working_dir}/samples/assets/vehicle_insurance/"
metadata_fname = "policy_5.json" # Change this to the file you want to evaluate
metadata_fpath = f"{path}{metadata_fname}"

with open(metadata_fpath, 'r') as f:
    data = json.load(f)
    
expected = VehicleInsurancePolicy(**data['expected'])
pdf_fname = data['fname']
pdf_fpath = f"{path}{pdf_fname}"

insurance_policy_evaluator = AccuracyEvaluator(match_keys=[])

## Extract data from the document

The following code block executes the data extraction process using Azure OpenAI's GPT-4o model using vision capabilities.

It performs the following steps:

1. Get the document bytes from the provided file path. _Note: In this example, we are processing a local document, however, you can use any document storage location of your choice, such as Azure Blob Storage._
2. Use pdf2image to convert the document's pages into images per page as base64 strings.
3. Using Azure OpenAI's GPT-4o model and its [Structured Outputs feature](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/structured-outputs?tabs=python-secure), extract a structured data transfer object (DTO) from the content of the images.

In [4]:
# Prepare the user content for the OpenAI API including any specific details for processing this type of document and the document page images.
user_content = []
user_content.append({
    "type": "text",
    "text": f"""Extract the data from this insurance policy. 
    - If a value is not present, provide null.
    - Some values must be inferred based on the rules defined in the policy.
    - Dates should be in the format YYYY-MM-DD."""
})

In [5]:
with Stopwatch() as image_stopwatch:
    document_bytes = open(pdf_fpath, "rb").read()
    page_images = convert_from_bytes(document_bytes)
    for page_image in page_images:
        byteIO = io.BytesIO()
        page_image.save(byteIO, format='PNG')
        base64_data = base64.b64encode(byteIO.getvalue()).decode('utf-8')
        
        user_content.append({
            "type": "image_url",
            "image_url": {
                "url": f"data:image/png;base64,{base64_data}"
            }
        })

In [6]:
with Stopwatch() as oai_stopwatch:
    completion = openai_client.beta.chat.completions.parse(
        model=settings.gpt4o_model_deployment_name,
        messages=[
            {
                "role": "system",
                "content": "You are an AI assistant that extracts data from documents.",
            },
            {
                "role": "user",
                "content": user_content
            }
        ],
        response_format=VehicleInsurancePolicy,
        max_tokens=4096,
        temperature=0.1,
        top_p=0.1,
        logprobs=True # Enabled to determine the confidence of the response.
    )

### Understanding the Structured Outputs JSON schema

Using [Pydantic's JSON schema feature](https://docs.pydantic.dev/latest/concepts/json_schema/), the [Insurance Policy](../../modules/vehicle_insurance_policy.py) data model is automatically converted to a JSON schema when applied to the `response_format` parameter of the OpenAI chat completions request.

The JSON schema is used to instruct the GPT-4o model to generate a strict output that adheres to the structure defined. The approach using Pydantic makes it easier for developers to manage the data structure in code, with helpful descriptions and examples that will be included in the final JSON schema.

Demonstrated below, you can see how the Insurance Policy data model is understood by the OpenAI request:

In [7]:
# Highlight the schema sent to the OpenAI model
print(json.dumps(VehicleInsurancePolicy.model_json_schema(), indent=2))

{
  "$defs": {
    "VehicleInsuranceCostDetails": {
      "description": "A class representing the cost details of a vehicle insurance policy.\n\nAttributes:\n    annual_total: The annual total cost of the vehicle insurance policy.\n    payable_by_date: The date by which the vehicle insurance policy must be paid.",
      "properties": {
        "annual_total": {
          "anyOf": [
            {
              "type": "number"
            },
            {
              "type": "null"
            }
          ],
          "description": "The annual total cost of the vehicle insurance policy.",
          "title": "Annual Total"
        },
        "payable_by_date": {
          "anyOf": [
            {
              "type": "string"
            },
            {
              "type": "null"
            }
          ],
          "description": "The date by which the vehicle insurance policy must be paid.",
          "title": "Payable By Date"
        }
      },
      "required": [
        "an

## Visualize the outputs

To provide context for the execution of the code, the following code blocks visualize the outputs of the data extraction process.

This includes:

- The accuracy of the structured data extraction comparing the expected output with the output generated by Azure OpenAI's GPT-4o model.
- The confidence score of the structured data extraction based on the log probability of the output generated by Azure OpenAI's GPT-4o model.
- The execution time of the end-to-end process.
- The total number of tokens consumed by the GPT-4o model.
- The side-by-side comparison of the expected output and the output generated by Azure OpenAI's GPT-4o model.

### Understanding Accuracy vs Confidence

When using AI to extract structured data, both confidence and accuracy are essential for different but complementary reasons.

- **Accuracy** measures how close the AI model's output is to a ground truth or expected output. It reflects how well the model's predictions align with reality.
  - Accuracy ensures consistency in the extraction process, which is crucial for downstream tasks using the data.
- **Confidence** represents the AI model's internal assessment of how certain it is about its predictions.
  - Confidence indicates that the model is certain about its predictions, which can be a useful indicator for human reviewers to step in for manual verification.

High accuracy and high confidence are ideal, but in practice, there is often a trade-off between the two. While accuracy cannot always be self-assessed, confidence scores can and should be used to prioritize manual verification of low-confidence predictions.

In [8]:
# Gets the parsed VehicleInsurancePolicy object from the completion response.
insurance_policy = completion.choices[0].message.parsed

expected_dict = expected.to_dict()
insurance_policy_dict = insurance_policy.to_dict()

In [9]:
# Determines the accuracy of the extracted data against the expected values.
accuracy = insurance_policy_evaluator.evaluate(expected=expected_dict, actual=insurance_policy_dict)

In [10]:
# Determines the confidence of the extracted data using the log probabilities of the completion response.
confidence = evaluate_confidence(insurance_policy_dict, completion.choices[0])

In [11]:
# Gets the total execution time of the data extraction process.
total_elapsed = image_stopwatch.elapsed + oai_stopwatch.elapsed

# Gets the prompt tokens and completion tokens from the completion response.
prompt_tokens = completion.usage.prompt_tokens
completion_tokens = completion.usage.completion_tokens

In [12]:
# Save the output of the data extraction result.
extraction_result = DataExtractionResult(insurance_policy_dict, confidence, accuracy, prompt_tokens, completion_tokens, total_elapsed)

with open(f"{working_dir}/samples/extraction/vision-based/openai.{pdf_fname}.json", "w") as f:
    f.write(extraction_result.to_json(indent=4))

In [13]:
# Display the outputs of the data extraction process.
df = pd.DataFrame([
    {
        "Accuracy": f"{accuracy['overall'] * 100:.2f}%",
        "Confidence": f"{confidence['_overall'] * 100:.2f}%",
        "Execution Time": f"{total_elapsed:.2f} seconds",
        "Image Pre-processing Execution Time": f"{image_stopwatch.elapsed:.2f} seconds",
        "OpenAI Execution Time": f"{oai_stopwatch.elapsed:.2f} seconds",
        "Prompt Tokens": prompt_tokens,
        "Completion Tokens": completion_tokens
    }
])

display(df)
display(get_extraction_comparison(expected_dict, insurance_policy_dict, confidence, accuracy['accuracy']))

,Accuracy,Confidence,Execution Time,Image Pre-processing Execution Time,OpenAI Execution Time,Prompt Tokens,Completion Tokens
0,92.31%,97.06%,78.25 seconds,4.23 seconds,74.02 seconds,9851,246


,Field,Expected,Extracted,Confidence,Accuracy
0,accident_excess_compulsory,300,300,99.86%,Match
1,accident_excess_unapproved_repair_penalty,250,250,93.16%,Match
2,accident_excess_voluntary,200,200,99.95%,Match
3,cost_annual_total,750.000000,750.000000,86.35%,Match
4,cost_payable_by_date,2024-05-22,None,0.00%,Mismatch
5,effective_from,2024-05-12,2024-05-12,99.99%,Match
6,effective_to,2025-05-11,2025-05-11,99.99%,Match
7,fire_and_theft_excess_compulsory,250,250,99.99%,Match
8,fire_and_theft_excess_unapproved_repair_penalty,250,250,99.34%,Match
9,fire_and_theft_excess_voluntary,150,150,99.98%,Match
